## Getting the Sistema de Informações sobre Mortalidade (SIM) dataset

### Firstly we're going to access the microdata columns and their data dictionary to understand the available features and select the most interesting to our exploration. 

In [1]:
import basedosdados as bd

In [44]:
# get the headers

sim_header = bd.read_sql(query=""" SELECT * FROM basedosdados.br_ms_sim.microdados LIMIT 0 """,
                          billing_project_id='explorando-basedosdados')
sim_columns = sim_header.columns
sim_columns

Downloading: 0rows [00:00, ?rows/s]


Index(['ano', 'sigla_uf', 'sequencial_obito', 'tipo_obito', 'causa_basica',
       'data_obito', 'hora_obito', 'naturalidade', 'data_nascimento', 'idade',
       'sexo', 'raca_cor', 'estado_civil', 'escolaridade', 'ocupacao',
       'codigo_bairro_residencia', 'id_municipio_residencia',
       'local_ocorrencia', 'codigo_bairro_ocorrencia',
       'id_municipio_ocorrencia', 'idade_mae', 'escolaridade_mae',
       'ocupacao_mae', 'quantidade_filhos_vivos', 'quantidade_filhos_mortos',
       'gravidez', 'gestacao', 'parto', 'obito_parto', 'morte_parto', 'peso',
       'obito_gravidez', 'obito_puerperio', 'assistencia_medica', 'exame',
       'cirurgia', 'necropsia', 'linha_a', 'linha_b', 'linha_c', 'linha_d',
       'linha_ii', 'circunstancia_obito', 'acidente_trabalho', 'fonte',
       'codigo_estabelecimento', 'atestante', 'data_atestado', 'tipo_pos',
       'data_investigacao', 'causa_basica_original', 'data_cadastro',
       'fonte_investigacao', 'data_recebimento', 'causa_basica_pre

In [33]:
# get dictionary to inform the column selection

sim_dic = bd.read_sql(query=""" SELECT coluna, chave, valor
                                FROM basedosdados.br_ms_sim.dicionario
                                WHERE id_tabela = 'microdados'""",
                          billing_project_id='explorando-basedosdados')
sim_dic.to_csv('../data/sim_dic')

# check if the time cover is unique
# set(sim_dic.cobertura_temporal)
sim_dic

Downloading: 100%|██████████| 557/557 [00:02<00:00, 199.67rows/s]


,coluna,chave,valor
0,tipo_obito,1,Fetal
1,tipo_obito,2,Não-fetal
2,naturalidade,1,ABU DABI
3,naturalidade,2,ACORES
4,naturalidade,3,AFEGANISTAO
...,...,...,...
552,local_ocorrencia,1,Hospital
553,local_ocorrencia,2,Outro estabelecimento de saúde
554,local_ocorrencia,3,Domicílio
555,local_ocorrencia,4,Via pública


## Tools for explore the data

In [124]:
# consult a column in the data dictionary

#%timeit sim_dic.query("coluna == 'tipo_obito'")
#%timeit sim_dic[ sim_dic['coluna'] == 'tipo_obito']
# Plain python is faster than .query. 


#sim_dic[ sim_dic['coluna'] == 'local_ocorrencia']
sim_dic[ sim_dic['coluna'].str.contains('alt_causa')]

,coluna,chave,valor


In [128]:
# Slicer to flick thougth the list of columns 
x = 9
sim_columns[10*x:(x*10 + 10)].to_list()

['fontes_informacao', 'alt_causa']

## Using the tools to slice 10 coluns and investigate their values on the dictionary we selected the columns of interest

In [131]:
# Manually selected columns of interest.
# todo: investigate col to refine selection

sim_selected_col_loose = {'ano',
                          'sigla_uf',
                          'causa_basica',
                          'data_obito',
                          'hora_obito',
                          'data_nascimento',
                          'idade'
                          'sexo',
                          'raca_cor',
                          'estado_civil',
                          'escolaridade',
                          'ocupacao',
                          'id_municipio_residencia',
                          'local_ocorrencia',
                          'codigo_bairro_ocorrencia',
                          'id_municipio_ocorrencia',
                          'idade_mae',
                          'escolaridade_mae',
                          'ocupacao_mae',
                          'quantidade_filhos_vivos',
                          'quantidade_filhos_mortos',
                          'peso',
                          'assistencia_medica',
                          'circunstancia_obito',
                          'acidente_trabalho',
                          'fonte',
                          'data_atestado',
                          'causa_basica_original',
                          'causa_basica_pre',
                          'id_municipio_svo_iml'
                          'causa_materna',
                          'status_do_epidem',
                          'status_do_nova',
                          'serie_escolar_falecido',
                          'serie_escolar_mae',
                          'escolaridade_2010',
                          'escolaridade_mae_2010',
                          'escolaridade_falecido_2010_agr'
                          'escolaridade_mae_2010_agr',
                          'diferenca_data',
                          'id_municipio_naturalidade',
                          'descricao_estabelecimento',
                          'alt_causa'}

sim_selected_col = {'ano',
                    'sigla_uf',
                    'causa_basica',
                    'data_obito',
                    'hora_obito',
                    'data_nascimento',
                    'idade'
                    'sexo',
                    'raca_cor',
                    'estado_civil',
                    'escolaridade',
                    'ocupacao',
                    'id_municipio_residencia',
                    'local_ocorrencia',
                    'codigo_bairro_ocorrencia',
                    'id_municipio_ocorrencia',
                    'idade_mae',
                    'escolaridade_mae',
                    'ocupacao_mae',
                    'quantidade_filhos_vivos',
                    'quantidade_filhos_mortos',
                    'peso',
                    'assistencia_medica',
                    'circunstancia_obito',
                    'acidente_trabalho',
                    'id_municipio_naturalidade'}


sim_col_investigate = set.difference(sim_selected_col_loose, sim_selected_col)
sim_col_investigate

{'alt_causa',
 'causa_basica_original',
 'causa_basica_pre',
 'data_atestado',
 'descricao_estabelecimento',
 'diferenca_data',
 'escolaridade_2010',
 'escolaridade_falecido_2010_agrescolaridade_mae_2010_agr',
 'escolaridade_mae_2010',
 'fonte',
 'id_municipio_svo_imlcausa_materna',
 'serie_escolar_falecido',
 'serie_escolar_mae',
 'status_do_epidem',
 'status_do_nova'}